# **TUTO**

`__name__` variable helps to control if you want to execute some part of your code if you're running the script directly and not from an import.

In [2]:
# Print the value of __name__ variable
print(__name__)

# Import var from dep.py
from script import var
print(var) # var = __name__ in the python file

# Check if __name__ returns 
if __name__ == "__main__":
    print("You are executing the main script")

# Check the __name__ returns from script.py
if var != "__main__":
    print(f"var has been imported from {var}.py")

__main__
script
You are executing the main script
var has been imported from script.py


Notes:
- If you don't init a logger/handler, logging.info() -or others levels- will automatically under the hood `logging.basicConfig()`
- The call to `logging.basicConfig()` with default setting with no logfile and logging level set to logging.warning (or 30) for **every loggers**
- **A handler will be init too and you won't be able to reinit the basic config without deleting existing handler or restarting the kernel**
- Logging use `sys.stderr` as main output file with higher priority than print file arg default `sys.stdout`

In [1]:
import logging
import sys
print("print use by default standard output file -sys.stdout- with a lower priority", file=sys.stdout) # Using flush is important here 
# Be sure you have restarted your kernel before executing this code
print(logging.getLogger().handlers)
print("Logging use the standard error output file -sys.stderr- with high priority", file=sys.stderr)
logging.warning('Watch out!')  # will print a message to the console
print(logging.getLogger().handlers)
logging.error("Non critical error")
logging.info('I told you so')  # will not print anything

Logging use the standard error output file -sys.stderr- with high priority
ERROR:root:Non critical error


print use by default standard output file -sys.stdout- with a lower priority
[]
[<StreamHandler stderr (NOTSET)>]


Notes:
- After a handler has been init, you are forced to delete them with assigning an empty list before being able to modify the basicConfig options
- You can use a logfile to track
- You can set the level using int or logging.level

In [9]:
import logging
# If you don't want to restart the kernel and modify some options, just set a empty list 
logging.getLogger().handlers = []
logging.basicConfig(
    filename='example.log',
    encoding='utf-8', # encoding style
    format="%(asctime)s %(levelname)s: %(message)s", # time + severty level + message
    datefmt='%d/%m/%Y %I:%M:%S %p', # time format
    filemode='a', # 'w' to write a new file, 'a' to append to the existing file
    level=30 # you can also pass the int level as 10 here. Be careful as it's the minimum value
    )
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')
logging.error('And non-ASCII stuff, too, like Øresund and Malmö')

In [2]:
# Get the level of severity for each level
for loglevel in ('debug', 'info', 'warning', 'error', 'critical'):
    print(loglevel.upper(), getattr(logging, loglevel.upper()))

DEBUG 10
INFO 20
WARNING 30
ERROR 40
CRITICAL 50


In [1]:
import logging
import script

# Afficher tous les loggers créés selon les imports
print("Loggers créés par les modules importés :")
for name, logger in logging.Logger.manager.loggerDict.items():
    print(f"- {name}")


2025-02-16 15:13:59,924 - script - DEBUG - Le logger de script.py a été initialisé.


Loggers créés par les modules importés :
- concurrent.futures
- concurrent
- asyncio
- tornado.access
- tornado
- tornado.application
- tornado.general
- stack_data.serializing
- stack_data
- parso
- prompt_toolkit.buffer
- prompt_toolkit
- parso.python.diff
- parso.python
- parso.cache
- Comm
- ipykernel.comm
- ipykernel
- IPKernelApp
- script


logging.basicConfig will overide every logger level.<br>
**Best practice is to set each logger manually than using basicConfig.**

In [ ]:
import logging
import script

# Using basicConfig set the level by default for EVERY loggers so you should set up each logger level manually
logger = logging.getLogger(__name__)
logger.setLevel(50)

# Check which logger level you have on script
script_logger = logging.getLogger("script")

# Check the level for each logger
for log in (script_logger, logger):
    print(log.name, log.level) # name attribute & level attribute


script 10
__main__ 50


# **EXEMPLE & CHEAT SHEET**

The logging library takes a modular approach and offers several categories of components: loggers, handlers, filters, and formatters.
- **Loggers** expose the interface that application code directly uses.
- **Handlers** send the log records (created by loggers) to the appropriate destination.
- **Filters** provide a finer grained facility for determining which log records to output.
- **Formatters** specify the layout of log records in the final output.

In [ ]:
import logging

# Création du logger
logger = logging.getLogger('mon_logger')
logger.setLevel(logging.DEBUG)  # Niveau général du logger

# Supprimer les anciens handlers si nécessaire
logger.handlers = []

# Formatters
formatter_console = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter_file = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Handler pour la console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)  # Tous les logs dans la console
console_handler.setFormatter(formatter_console)

# Ajout d'un filtre pour le StreamHandler
class KeywordFilter(logging.Filter): # Hérite de la classe Filter
    def filter(self, record): # Objet record instancié par Logging
        return 'ERROR' in record.getMessage()  # Filtrer uniquement les messages contenant 'ERROR'

console_handler.addFilter(KeywordFilter())

# Handler pour un fichier
file_handler = logging.FileHandler('mon_log.log')
file_handler.setLevel(logging.WARNING)  # Seulement les WARNING et plus graves dans le fichier
file_handler.setFormatter(formatter_file)

# Ajouter les handlers au logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Exemples de logs
logger.debug("C'est un message DEBUG")
logger.info("C'est un message INFO")
logger.warning("C'est un message WARNING")
logger.error("C'est un message ERROR")


L'objet `record` passé au **filter** dans le système de `logging` de Python est une instance de la classe `LogRecord`. Cet objet contient toutes les informations relatives à l'événement de log qui vient d'être émis. Il est utilisé pour transmettre des données supplémentaires sur le message de log, telles que le niveau de sévérité, la source du message, le message lui-même, etc.

### Attributs de `LogRecord` :

Voici quelques attributs clés que tu peux utiliser sur un objet `record` :

1. **`record.getMessage()`** : Retourne le message de log, après que les arguments ont été formatés. C'est le texte du message que tu passes à la méthode `logger.log()` (par exemple, `"C'est un message d'erreur"`).

2. **`record.levelname`** : Le nom du niveau de log (par exemple, `'DEBUG'`, `'INFO'`, `'WARNING'`, `'ERROR'`, `'CRITICAL'`).

3. **`record.levelno`** : Le niveau du message sous forme numérique (par exemple, `10` pour `DEBUG`, `20` pour `INFO`, `30` pour `WARNING`, etc.).

4. **`record.name`** : Le nom du logger. Cela peut être utile si tu utilises plusieurs loggers avec des noms différents.

5. **`record.pathname`** : Le chemin absolu du fichier source à partir duquel le message de log a été émis.

6. **`record.filename`** : Le nom du fichier source (sans le chemin) à partir duquel le message de log a été émis.

7. **`record.module`** : Le nom du module (ou du fichier) où le message de log a été généré.

8. **`record.funcName`** : Le nom de la fonction où le message a été émis.

9. **`record.lineno`** : Le numéro de ligne du fichier source où le message de log a été émis.

10. **`record.exc_info`** : Si une exception a été levée, ce champ contiendra un tuple `(type, value, traceback)` de l'exception.

11. **`record.stack_info`** : Si un traceback est disponible, cette information contient un objet représentant les informations de la pile (par exemple, dans un log de niveau `ERROR`).

12. **`record.args`** : Si des arguments ont été passés au message de log (par exemple, avec un formatage comme `logger.debug("Un nombre : %d", 5)`), cet attribut contiendra ces arguments.

13. **`record.created`** : L'heure (en secondes depuis l'époque) à laquelle l'enregistrement du log a été créé.

14. **`record.msecs`** : Les millisecondes de l'heure à laquelle l'enregistrement du log a été créé.

15. **`record.relativeCreated`** : Le temps écoulé depuis que le processus a démarré, au moment où l'enregistrement a été créé.

16. **`record.thread`** : L'identifiant du thread qui a émis le message de log.

17. **`record.threadName`** : Le nom du thread qui a émis le message de log.

18. **`record.process`** : L'identifiant du processus qui a émis le message de log.

19. **`record.processName`** : Le nom du processus qui a émis le message de log.

### Méthodes supplémentaires :
En plus de `getMessage()`, tu peux aussi manipuler directement l'objet `record` avec des méthodes comme :
- **`record.getMessage()`** : Retourne le message de log formaté.
- **`record.exc_info`** : Permet de récupérer ou d'ajouter des informations sur une exception (utile pour le logging d'exception).


**Import logging config from a config file is a good pratice & easier to centralize.**

In [ ]:
import logging.config

logging.config.fileConfig('logging.conf')